In [4]:
from huggingface_hub import login
login()

In [4]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments
import torch

# ----------------------------
# Load dataset
# ----------------------------
dataset = load_dataset("text", data_files={"train": "syrian.txt"})
print(dataset["train"][0])

# ----------------------------
# Model & tokenizer (JAIS)
# ----------------------------
model_name = "inceptionai/Jais-2-8B-Chat"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# JAIS does not always define pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# For training
model.gradient_checkpointing_enable()
model.config.use_cache = False

model.config.pad_token_id = tokenizer.pad_token_id

# ----------------------------
# Tokenization
# ----------------------------
def tokenize_and_add_labels(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding=False
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(
    tokenize_and_add_labels,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# ----------------------------
# LoRA configuration
# ----------------------------
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ----------------------------
# Training arguments
# ----------------------------
training_args = TrainingArguments(
    output_dir="./syrian_dialect_jais_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=5,
    logging_steps=1000,
    save_strategy="steps",
    save_steps=2000,
    fp16=True,
    save_total_limit=2,
    report_to="none"
)

# ----------------------------
# Trainer
# ----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

trainer.train()


{'text': ' قامت على طاولتها واخدت الشنته وطلعت مع الكفتيريا '}


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

trainable params: 13,631,488 || all params: 8,104,032,768 || trainable%: 0.1682


TrainOutput(global_step=31420, training_loss=3.504646640245813, metrics={'train_runtime': 21471.8192, 'train_samples_per_second': 5.853, 'train_steps_per_second': 1.463, 'total_flos': 1.2420011937755136e+17, 'train_loss': 3.504646640245813, 'epoch': 5.0})

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_id = "inceptionai/Jais-2-8B-Chat"
lora_path = "./syrian_dialect_jais_lora/checkpoint-31420"
merged_output_path = "./jansSyrianCou"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, lora_path)

# Merge LoRA weights into base model
model = model.merge_and_unload()

# Save merged model
model.save_pretrained(merged_output_path, safe_serialization=True)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained(lora_path)
tokenizer.save_pretrained(merged_output_path)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('./jansSyrianCou/tokenizer_config.json', './jansSyrianCou/added_tokens.json')

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Path to merged model
model_path = "./jansSyrianCou"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load merged model
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="cuda"
)

model.eval()

# Test prompt (Syrian dialect example)
prompt = "Hello! Can you tell me a fun fact about space?"

# Tokenize
inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to(model.device)

# Generate
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )

# Decode
output_text = tokenizer.decode(
    output_ids[0],
    skip_special_tokens=True
)

print("=== Model Output ===")
print(output_text)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

=== Model Output ===
Hello! Can you tell me a fun fact about space?t זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp君?ᵈt זp


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch

# Paths
base_model_name = "inceptionai/Jais-2-8B-Chat"
lora_model_path = "./syrian_dialect_jais_lora/checkpoint-31420"
merged_model_path = "./jais_2_8b_syrian_merged"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load LoRA adapter
model = PeftModel.from_pretrained(model, lora_model_path)

# Merge LoRA weights into the base model
model = model.merge_and_unload()  # This merges LoRA into base

# Save merged model
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

print(f"Merged model saved to {merged_model_path}")

# ===============================
# Test the merged model
# ===============================
# Load the merged model


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Merged model saved to ./jais_2_8b_syrian_merged


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Setting `pad_token_id` to `eos_token_id`:150024 for open-end generation.


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Path to the merged model
merged_model_path = "./jais_2_8b_syrian_merged"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

# Load merged model
model = AutoModelForCausalLM.from_pretrained(
    merged_model_path,
    device_map="auto",
    torch_dtype=torch.float16
)

# Test prompt
prompt = "شو القصة"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Remove token_type_ids if present
if "token_type_ids" in inputs:
    del inputs["token_type_ids"]

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7
    )

response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Model response:", response)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:150024 for open-end generation.


Model response: شو القصة شو هاد اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار معكن اديش صار
